<a href="https://colab.research.google.com/github/jtao22/PythonAI/blob/main/Weather/AustraliaRainPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
#import 
import numpy as np
import pandas as pd 
from sklearn.preprocessing import LabelEncoder as LE
from sklearn.preprocessing import MinMaxScaler as MMS
from sklearn.model_selection import train_test_split as tts
from sklearn.linear_model import LogisticRegression as LR
from sklearn.neighbors import KNeighborsClassifier as KNC
from sklearn.svm import SVC 
from sklearn.metrics import accuracy_score as acc

In [ ]:
from google.colab import files
files.upload()

In [80]:
data = pd.read_csv('rain.csv')
data.shape

(145460, 23)

In [81]:
#data cleaning 
data = data.fillna(data.mean())
data = data.dropna()
data.drop_duplicates(inplace = True)
data.isnull().sum()

Date             0
Location         0
MinTemp          0
MaxTemp          0
Rainfall         0
Evaporation      0
Sunshine         0
WindGustDir      0
WindGustSpeed    0
WindDir9am       0
WindDir3pm       0
WindSpeed9am     0
WindSpeed3pm     0
Humidity9am      0
Humidity3pm      0
Pressure9am      0
Pressure3pm      0
Cloud9am         0
Cloud3pm         0
Temp9am          0
Temp3pm          0
RainToday        0
RainTomorrow     0
dtype: int64

In [82]:
#parsing date into day, month, year
data['Date']= pd.to_datetime(data["Date"])
data['month'] = (data.Date.dt.month)/12
data['day'] = (data.Date.dt.day)/31
data = data.drop(columns=['Date'])
data.head(5)
data.dtypes

Location          object
MinTemp          float64
MaxTemp          float64
Rainfall         float64
Evaporation      float64
Sunshine         float64
WindGustDir       object
WindGustSpeed    float64
WindDir9am        object
WindDir3pm        object
WindSpeed9am     float64
WindSpeed3pm     float64
Humidity9am      float64
Humidity3pm      float64
Pressure9am      float64
Pressure3pm      float64
Cloud9am         float64
Cloud3pm         float64
Temp9am          float64
Temp3pm          float64
RainToday         object
RainTomorrow      object
month            float64
day              float64
dtype: object

In [83]:
#one-hot encoding and min-max scaling
le = LE()
Y = data['RainTomorrow']
Y = le.fit_transform(Y)
data = data.drop(columns=['RainTomorrow'])
for col in data:
  if (data[col].dtypes == np.float64):
    maxim = np.max(data[col])
    minim = np.min(data[col])
    data[col] = (data[col]-minim)/(maxim-minim)
  else:
    data[col] = le.fit_transform(data[col])
data = data.values

In [84]:
trainX,testX,trainY,testY = tts(data,Y,test_size=0.2, random_state=42)

In [86]:
#create models
#linear regression
lr=LR()
lr.fit(trainX,trainY)
#K Neighbors Classifier
knc=KNC(n_neighbors=8)
knc.fit(trainX,trainY)
#Support Vector Machine
svc = SVC()
svc.fit(trainX,trainY)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [87]:
#predict
lpredict=lr.predict(testX)
kpredict = knc.predict(testX)
spredict = svc.predict(testX)
print(acc(testY,lpredict))
print(acc(testY,kpredict))
print(acc(testY,spredict))

0.8416053673914801
0.8022795246948509
0.8286314768409991
